In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.environ["WANDB_API_KEY"] = "0" ## to ıgnore warning

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# EDA

In [2]:

train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [3]:
# set GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")  # Создаем стратегию для одного GPU
    except RuntimeError as e:
        print(e)
else:
    strategy = tf.distribute.get_strategy()
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [4]:
train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [5]:
len(train)

12120

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12120 entries, 0 to 12119
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          12120 non-null  object
 1   premise     12120 non-null  object
 2   hypothesis  12120 non-null  object
 3   lang_abv    12120 non-null  object
 4   language    12120 non-null  object
 5   label       12120 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 568.2+ KB


In [7]:
train.isnull().sum()

id            0
premise       0
hypothesis    0
lang_abv      0
language      0
label         0
dtype: int64

In [8]:
!pip install plotly

In [9]:
import plotly.express as px

# Define the data (example data)
labels = ['English', 'French', 'Spanish']
frequencies = [45, 30, 25]  # Example frequencies for each language

# Define a custom color map for the labels
color_map = {
    'English': 'rgb(31, 119, 180)',  # Specify your desired color for 'English'
    'French': 'rgb(255, 127, 14)',   # Specify your desired color for 'French'
    'Spanish': 'rgb(44, 160, 44)',   # Specify your desired color for 'Spanish'
    # Add more label-color mappings as needed
}

fig = px.pie(
    values=frequencies,
    names=labels,
    title='Languages distribution',
    color_discrete_map=color_map  # Use the custom color map here
)

# Increase the size of the pie chart
fig.update_layout(width=800, height=600)

fig.show()


In [10]:
import plotly.graph_objects as go

train['text_length'] = train['premise'].apply(len)

fig = go.Figure(data=[go.Histogram(x=train['text_length'], 
                                   nbinsx=50,
                                   marker_color='skyblue')])
fig.update_layout(title_text='Text  distribution in the context of "premise"', # title of plot
                  xaxis_title_text='Len of text in premise', # xaxis label
                  yaxis_title_text='Number of sentences', # yaxis label
                  bargap=0.2, # gap between bars of adjacent location coordinates
                  bargroupgap=0.1) # gap between bars of the same location coordinates
fig.show()

In [11]:
train['text_length'] = train['hypothesis'].apply(len)

fig = go.Figure(data=[go.Histogram(x=train['text_length'], 
                                   nbinsx=50,
                                   marker_color='skyblue')])
fig.update_layout(title_text='Text  distribution in the context of "Hypothesis"', # title of plot
                  xaxis_title_text='Len of text in hypothesis', # xaxis label
                  yaxis_title_text='Number of sentences', # yaxis label
                  bargap=0.2, # gap between bars of adjacent location coordinates
                  bargroupgap=0.1) # gap between bars of the same location coordinates
fig.show()

#  Setup

In [12]:
!pip install evaluate # library for metrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.7 MB/s eta 0:00:00


In [13]:
import evaluate
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer

In [14]:
model_name = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
train = train.drop(labels=['language', 'text_length', 'lang_abv'], axis=1)
test = test.drop(labels=['language','lang_abv'], axis=1)

In [16]:
from datasets import Dataset, DatasetDict

In [17]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()
ds['train'] = train_ds
ds['validation'] = val_ds
ds['test'] = test_ds

ds

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', '__index_level_0__'],
        num_rows: 9696
    })
    validation: Dataset({
        features: ['id', 'premise', 'hypothesis', 'label', '__index_level_0__'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis'],
        num_rows: 5195
    })
})

In [18]:
def tokenizer_sentence(data):
    return tokenizer(data['premise'], data['hypothesis'], truncation=True)

In [19]:
tokenized_ds = ds.map(tokenizer_sentence, batched=True)


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [20]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Let's Build Model

In [21]:
import torch.nn as nn
from transformers import XLMRobertaModel

In [22]:
class CustomXLMRobertaModel(nn.Module):
    def __init__(self, num_labels):
        super(CustomXLMRobertaModel, self).__init__()
        model_name = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
        self.roberta = XLMRobertaModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Sequential(
            nn.Linear(768, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_labels)
        )
        self.loss = nn.CrossEntropyLoss()
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        output = self.dropout(output.pooler_output)
        logits = self.classifier(output)

        if labels is not None:
            loss = self.loss(logits.view(-1, self.num_labels), labels.view(-1))
            return {"loss": loss, "logits": logits}
        else:
            return logits

In [23]:
model = CustomXLMRobertaModel(num_labels=3) # we have 3 classes

Some weights of the model checkpoint at symanto/xlm-roberta-base-snli-mnli-anli-xnli were not used when initializing XLMRobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at symanto/xlm-roberta-base-snli-mnli-anli-xnli and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to us

# Train Model

In [24]:
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_metric

training_args = TrainingArguments("/content",
                                  optim="adamw_torch",
                                  num_train_epochs=5,
                                  evaluation_strategy="epoch",
                                  logging_dir='./logs',
                                  logging_steps=10)

f1_metric = load_metric("f1")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_metric.compute(predictions=predictions, references=labels, average="micro")
    }


Trainer provides a straightforward and fast way to train and evaluate your model.

trainer = Trainer(...: This creates an instance of the Trainer class, which will be used for training and evaluating the model. The arguments passed are:

model: this is the model to be trained. In this context, it isn't defined yet, but it could be any model compatible with the transformers library.

args=training_args: these arguments control the training process. training_args should be an instance of TrainingArguments or compatible, which defines parameters like the learning rate, batch size, etc.

train_dataset=tokenized_ds["train"] and eval_dataset=tokenized_ds["validation"]: these are the datasets for training and evaluating the model. In this case, they are taken from the dictionary tokenized_ds, presumably containing tokenized versions of the original data.

data_collator=data_collator: the data_collator is a function that takes a list of samples from the Dataset and collates them into mini-batches (batches) for training or evaluation. It isn't defined in this particular context.

tokenizer=tokenizer: this is the tokenizer that was used to tokenize the input data.

compute_metrics=compute_metrics: this is a function that will be used to compute metrics during evaluation. In this context, it isn't defined yet. It should take the outputs from Trainer.evaluate() and return a dictionary where keys are the names of the metrics and values are the computed metrics.

In [25]:
from transformers import Trainer

trainer = Trainer(
    model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # передаем функцию compute_metrics
)

# Let's Setup env

Wandb, short for Weights & Biases, is a machine learning tool designed to monitor and visualize the progress of model training and facilitate the comparison of different experiments. It offers a user-friendly web interface that enables real-time observation of experiments, displays graphs depicting metrics like loss and accuracy, allows for the storage and retrieval of model weights, and permits easy sharing of results with collaborators.

By setting the environment variable `os.environ["WANDB_DISABLED"] = "true"`, you can disable the integration with Weights & Biases. This can be beneficial if you prefer to avoid sending your experiments to Wandb or if you are working in an environment with restricted or no internet access. By doing so, you prevent any interactions or data exchange with Wandb, preserving the experiments within your local environment.

In [26]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [28]:
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.491800,0.574225,0.795380,{'f1': 0.7953795379537953}
2,0.503700,0.559343,0.804868,{'f1': 0.8048679867986799}
3,0.411400,0.877657,0.798680,{'f1': 0.7986798679867986}


Trainer is attempting to log a value of "{'f1': 0.7953795379537953}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8048679867986799}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7986798679867986}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2759 in training_step            │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2784 in compute_loss             │
│                                                                                                  │
│   2781 │   │   │   labels = inputs.pop("labels")                                                 │
│   2782 │   │   else:                                       

# Get Model predictions

In [29]:
predictions = trainer.predict(tokenized_ds["test"])
predictions

PredictionOutput(predictions=array([[-2.7174554, -2.9699578,  4.2133455],
       [-3.0694995,  4.623986 , -3.014065 ],
       [ 4.928742 , -1.8966272, -2.1906483],
       ...,
       [ 4.900323 , -1.946845 , -1.9885854],
       [ 4.8683796, -1.8874103, -1.9761918],
       [-3.0400996, -2.7672114,  4.3198524]], dtype=float32), label_ids=None, metrics={'test_runtime': 644.3099, 'test_samples_per_second': 8.063, 'test_steps_per_second': 1.009})

In [30]:
logits = torch.from_numpy(predictions.predictions)
probs = torch.softmax(logits, -1).tolist() # convert to probability
probs[:5]

[[0.0009755239589139819, 0.000757839996367693, 0.9982665777206421],
 [0.00045535998651757836, 0.9990634322166443, 0.00048131527728401124],
 [0.9981083869934082, 0.001083821291103959, 0.000807729025837034],
 [0.0005035892245359719, 0.9989799857139587, 0.000516395375598222],
 [0.003490425180643797, 0.995607316493988, 0.0009022981976158917]]

In [31]:
outputs = []

for index, prob in enumerate(probs):
    # ind indx with max probability of class
    predicted_label = prob.index(max(prob))
    element_id = ds['test']['id'][index]
    prediction = (element_id, predicted_label)
    outputs.append(prediction)

# Save Submision¶


In [32]:
submission = pd.DataFrame(outputs, columns=['id', 'prediction'])
submission.to_csv("submission.csv", index=False)
submission.head()

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1


Get Model predictions